In [ ]:
import numpy as np                    # NumPy is imported under the alias as np 
import pandas as pd                   # pandas is imported under the alias as pd 
import os                             # os is imported (provides functions for interacting with the operating system)
import matplotlib.pyplot as plt       # matplotlib.pyplot is imported under the alias plt
import tensorflow as tf               # tenserflow is imported under the alias tf
import cv2                            # By using it, one can process images and videos to identify objects and faces
from tensorflow.keras.utils import to_categorical            #Converts a class vector (integers) to binary class matrix.
from tensorflow.keras.preprocessing.image import load_img, img_to_array       #load_img (Loads an image into PIL format) and img_to_array (Converts a PIL Image instance to a Numpy array).
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator    #ImageDataGenerator Generates batches of tensor image data with real-time data augmentation.  
from sklearn.metrics import classification_report, log_loss, accuracy_score   #classification_report(is used to show the precision, recall and F1 Score). log_loss(returns y_pred probabilities for its training data y_true). accuracy_score(computes subset accuracy)
from sklearn.model_selection import train_test_split         #train_test_split (splits data arrays into two subsets: for training data and for testing data)
from tqdm import tqdm                 #tqdm is a library that allows you to output a smart progress bar by wrapping around any iterable
import random                         #random imports the random module which contains a variety of things to do with random number generation.

In [ ]:
data_dir = '../input/animal-image-dataset-90-different-animals/animals/animals'
#storing our dataset in a data directory named data_dir.

In [ ]:
Name = os.listdir(data_dir) #storing list of all keys of data_dir directory in 'Name' variable,
                            # here 'Name' stores names of all animals in our data set
print(Name)                 #printing the data of given data set. here, the data is all the names of animals taken in the project.
print(len(Name))            #printing length of data. which is number of animals taken. 
my_list = os.listdir(data_dir)
my_list.sort()

In [ ]:
N=list(range(len(Name)))             #len(name) is the length of our dataset, which is total number of animals.
                                     #range(len(name)) is the integers from 0 to p-1 (where p is len(name), also total no of animals)
                                     #list(range(len(Name))) is list of integers from 0 to p-1
                                     #here N is a list where values(integers from 0 to p-1) are stored 

normal_mapping=dict(zip(Name,N))     #dict(zip(Name,N)) is the dictionary that maps keys(names of animals) of Name with values(integers from 0 to p-1) of N. 
reverse_mapping=dict(zip(N,Name))    #reverse mapping  

In [ ]:
# datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
#                         width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

datagen = ImageDataGenerator(rotation_range=20,      # rotation_range=20 signifies 20 degrees of image rotation        
                             shear_range=10,          #shear_range=10 implies shearing transformation 0f 10
                             validation_split=0.2)    #validation_split=0.2 signifies 0.2 fraction of the training data to be used as validation data

train_generator = datagen.flow_from_directory(data_dir,                #flow_from_directory() allows to read the images directly from the directory data_dir            
                                              target_size=(224,224),   #imaze size will be resized to the size of 224 units of height and 224 units of width
                                              subset='training')       #training images will be augmented

val_generator = datagen.flow_from_directory(data_dir,                  #flow_from_directory() allows to read the images directly from the directory data_dir 
                                            target_size=(224,224),     #imaze size will be resized to the size of 224 units of height and 224 units of width
                                            subset='validation')       #validation images will be augmented

In [ ]:
pretrained_model3 = tf.keras.applications.VGG19(input_shape=(224,224,3),include_top=False,weights='imagenet',pooling='avg')
#instantiating the VGG19 model. image height of 224 units, width of 224 units and a total of 3 channels.
#include_top=False indicates that it does not include the 3 fully-connected layers at the top of the network
#weights='imagenet' indicates pre-training on ImageNet. pooling='avg' means that global average pooling will be applied to the output
pretrained_model3.trainable = False    #freezes all the weights and starts compilation
#Transfer learning 

In [ ]:
inputs3 = pretrained_model3.input          #instantiating model
x3 = tf.keras.layers.Dense(224, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(len(Name), activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)  
#starting from Input, we chain layer calls to specify the model's forward pass, 
#      and finally creating our model from inputs and outputs

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#here the loss function is taken as 'categorical_crossentropy', optimizer as 'adam' and metrics as 'accuracy'
#here, we are compiling the model

In [ ]:
his=model.fit(train_generator,validation_data=val_generator,epochs=100)
#measuring how well the model generalizes to trained data by caluculating losses and accuracy.

In [ ]:
image = cv2.imread('../input/animal-image-dataset-90-different-animals/animals/animals/badger/00f56e5c92.jpg')
#reads the image of antelope. which is 02f4b3be2d.jpg
#../input/animal-image-dataset-90-different-animals/animals/animals/badger/00f56e5c92.jpg

In [ ]:
def predict (img,model):            #defining predict function and passing variables img, model
    img = cv2.resize(img,dsize=(224,224),interpolation=cv2.INTER_CUBIC)     #resizing the image of antelope
    img = np.expand_dims(img, axis = 0)       #Expanding the shape of img array and Inserting a new axis that will appear at the axis=0 position in the expanded array shape.
    prediction = model.predict(img)           #predicts img and returns the learned label for each object in the array
    cname =np.argmax(prediction)             #Returns the indices of the maximum values of prediction
    
    animal = my_list[cname]                  #normal mapping
    return animal                             #returns the value, animal to the function.
    
#normal_mapping['antelope']                 

#here, in this cell we resize and predict the img.then extract maximum values along axis=0 and return them after normal mapping.
    

In [ ]:
predict(image,model) # sending the image of antelope and model data through predict function